The data is from the hiPSC single cell image dataset from the Allen Institute for Cell Science: https://open.quiltdata.com/b/allencell/packages/aics/hipsc_single_cell_image_dataset. Original publication of the data: 

Viana, M.P., Chen, J., Knijnenburg, T.A. et al. Integrated intracellular organization and its variations in human iPS cells. Nature 613, 345–354 (2023). https://doi.org/10.1038/s41586-022-05563-7

This demo will gather data to demonstrate how to train a model to generate 2D synthetic microscopy images from binary masks, using dataset: nucleophosmin (NPM1).

In [ ]:
# you need quilt3 package to download the data:
! pip install quilt3

In [1]:
import pandas as pd
import quilt3
from pathlib import Path
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from random import random, sample
from shutil import move
import numpy as np

In [2]:
# we use the NPM1 and HIST1H2BJ cell lines for this demo
cline = "NPM1"

# set up path
parent_path = Path("../../data/synthetic2D")

raw_path = parent_path / Path("download")
raw_path.mkdir(exist_ok=True)
train_path = parent_path / f"{cline}" / Path("train")
train_path.mkdir(exist_ok=True, parents=True)
holdout_path = parent_path / f"{cline}"/ Path("holdout")
holdout_path.mkdir(exist_ok=True, parents=True)

In [3]:
# connect to quilt and load meta table
pkg = quilt3.Package.browse(
    "aics/hipsc_single_cell_image_dataset", registry="s3://allencell"
)
meta_df_obj = pkg["metadata.csv"]
meta_df_obj.fetch(parent_path / "meta.csv")
meta_df = pd.read_csv(parent_path / "meta.csv")

# fetch the data of the specific cell line
meta_df_line = meta_df.query("structure_name==@cline")

# collapse the data table based on FOVId
meta_df_line.drop_duplicates(subset="FOVId", inplace=True)

# reset index
meta_df_line.reset_index(drop=True, inplace=True)

Loading manifest: 100%|██████████| 484465/484465 [00:12<00:00, 38.3k/s]
100%|██████████| 1.69G/1.69G [00:43<00:00, 39.2MB/s] 
/tmp/ipykernel_3540014/3370391117.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(parent_path / "meta.csv")
/tmp/ipykernel_3540014/3370391117.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_df_line.drop_duplicates(subset="FOVId", inplace=True)


In [5]:
# download the images and segmentation
# we only need a small amount of data for the purpose of demonstration
num_of_sample = 400
tmp_list = []
for row in meta_df_line.itertuples():
    if row.Index > num_of_sample:
        break
    # fetch the raw image (multi-channel)
    subdir_name = row.fov_path.split("/")[0]
    file_name = row.fov_path.split("/")[1]

    local_fn = raw_path / f"{row.FOVId}_original.tiff"
    pkg[subdir_name][file_name].fetch(local_fn)

    # extract the structure channel
    reader = AICSImage(local_fn)
    img = reader.get_image_data("ZYX", C=row.ChannelNumberStruct, S=0, T=0)

    subdir_name = row.struct_seg_path.split("/")[0]
    file_name = row.struct_seg_path.split("/")[1]

    local_fn = raw_path / f"{row.FOVId}_seg.tiff"
    pkg[subdir_name][file_name].fetch(local_fn)

    # extract the structure segmentation
    reader = AICSImage(local_fn)
    seg = reader.get_image_data("ZYX", C=0, S=0, T=0).astype(np.uint8)
    seg[seg > 0] = 1

    if random() < 0.2:
        out_path = holdout_path
    else:
        out_path = train_path

    # find mid z
    z_range = np.where(np.any(seg > 0, axis=(1, 2)))
    z_range = z_range[0]
    mid_z = (z_range[-1] + z_range[0]) // 2
    mid_z = int(mid_z)

    # save the data
    im_fn = out_path / f"{row.FOVId}_GT.tiff"
    seg_fn = out_path / f"{row.FOVId}_IM.tiff"
    OmeTiffWriter.save(img[mid_z, :, :], im_fn, dim_order="YX")
    OmeTiffWriter.save(seg[mid_z, :, :], seg_fn, dim_order="YX")

100%|██████████| 394M/394M [00:15<00:00, 26.2MB/s]  
/mnt/eternus/anaconda3_shared/envs/jc_im2im_dev/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
100%|██████████| 192k/192k [00:03<00:00, 61.0kB/s]
/mnt/eternus/anaconda3_shared/envs/jc_im2im_dev/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/mnt/eternus/anaconda3_s

In [6]:
# you may remove the download folder now.
from shutil import rmtree
import os
rmtree(raw_path)
os.remove(parent_path / "meta.csv")
